In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import json


In [ ]:
def convert_geotiff_to_png(geotiff_path, output_png_path, output_metadata_path, threshold=32760):
    with rasterio.open(geotiff_path) as src:
        data = src.read(1)  # Read the first band
        
        # Get metadata for georeferencing
        metadata = {
            "bounds": src.bounds._asdict(),
            "width": src.width,
            "height": src.height,
            "crs": src.crs.to_string()
        }
        
        mask = data > threshold
        valid_data = np.ma.masked_array(data, mask)
        vmin = np.nanmin(valid_data)
        vmax = np.nanmax(valid_data[valid_data <= threshold])
        norm_data = colors.Normalize(vmin=vmin, vmax=vmax)

        dpi = 360
        width_inches = src.width / dpi
        height_inches = src.height / dpi

        # Create a masked version where values > threshold will be transparent
        cmap = plt.cm.viridis.copy()
        cmap.set_bad(alpha=0)  # Set masked values to be transparent
        
        fig = plt.figure(figsize=(width_inches, height_inches), dpi=dpi)
        ax = plt.Axes(fig, [0, 0, 1, 1])  # No margins
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(valid_data, cmap=cmap, norm=norm_data, interpolation="nearest", aspect="auto")
        plt.savefig(
            output_png_path, 
            dpi=dpi,
            bbox_inches="tight", 
            pad_inches=0, 
            transparent=True
        )
        plt.close(fig)
        
        # Save the metadata as JSON
        with open(output_metadata_path, "w") as f:
            json.dump(metadata, f)
        
        print(f"Converted {geotiff_path} to {output_png_path} with metadata at {output_metadata_path}")

# Example usage
convert_geotiff_to_png("data/GOSIF_2020.M06.tif", "data/GOSIF_2020.M06.png", "data/GOSIF_2020.M06_metadata.json")

In [ ]:
from IPython.display import IFrame

# gosif.html has to be loaded with live server (To Do: find a better way to do this)
IFrame(src="http://localhost:5500/notebooks/gosif.html", width=1200, height=800)